This is run a Google Colab Notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Downloading files

In [ ]:
# !wget --header="Host: www.ncbi.nlm.nih.gov" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-GB,en;q=0.9,en-US;q=0.8,bn;q=0.7" --header="Referer: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE214611" --header="Cookie: _gid=GA1.2.843222926.1683780721; ncbi_sid=0C4234AB45C74711_0000SID; _ga=GA1.1.1244755766.1683780721; _ga_DP2X732JSX=GS1.1.1683780720.1.1.1683782404.0.0.0; _gat_ncbiSg=1; ncbi_pinger=N4IgDgTgpgbg+mAFgSwCYgFwgAwGEAsATAMwDsAjAGz4Csup+pl52rb7H5IANCAMYAbZHwDWAOygAPAC6ZQxAJyYQVUkwAc2HiHxasqhfirb8+ZYOHips3rWVdblZQDMAhgIDOUE6WXUttur25PiUAHSUkZTqpNjk5BTqQbZKWNo0eiCI0tJgHhgA9AUA7qVhYnwARsjlAgC25ciIYQDmAPYwBS1QbQWobcViAm2uqAUA/K58fAC8AOIAygCihCHM5ABkzm0Qda7SM87IAt68NFxpZ4TKABTZuQCUIAC+vACuQyOoEjJyIMTEFzuLzaYhmLDSCBvU7/IKXf6pFSUNSUQwmTIGIhKWzgkAfYajH42HQ0ZTXM5OLA0VHpXxYAIgZjKTSA178Np1OptMREv7XLDdNqg5QmFxvAQCdIi3iETJ4IhkKi0eiMZgcdXsBwgVbKQUYAByAHl9UsMM5xQIDcaltpCPyQKViuUqjUhg0xE1Wh1bYjyApZaCMdhNKCLip1Phkv97SwaIDbID6WFiOowuSQHH7CR0zRcdTfM9nkA" --header="Connection: keep-alive" "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE214611&format=file" -c -O '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW.tar'

In [ ]:
# !tar -xvf '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW.tar' -C '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/'

In [ ]:
# !cd '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/' && for f in *.tar.gz; do tar -xvzf "$f" -C "$(dirname "$f")"; done

In [ ]:
# !unzip '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping.zip' -d '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW'

## scPSS

In [2]:
!pip install scanorama
!pip install git+https://github.com/SaminRK/scPSS.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553317 sha256=81c36e1f88c074a4fede43d3a4913659075fca5d175f6841c2a310cd3b47dc41
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=3663000423e3a8f9db75f10cce1d9d341cfaa3a9f9c1d3493572bf2fbec073c0
  Stored in directory: /root/.cache/pip/wheels/71/ae/f7/5b7e1e16a133d094431428169e9a8d9f750afe5459f0333c9b
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=dc69863da9b74e0e551887f30380ec445d1242723efc7408a2d0977d3368135a
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
S

In [3]:
## Modify to the source directory of single cell data
ROOT_PATH = '/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/data/Infracted Heart Border Zone - GSE214611_RAW'

In [4]:
import scanpy as sc
import pandas as pd

In [5]:
metadata_sc = pd.read_csv(f'{ROOT_PATH}/Single-Cell-Spatial-Transcriptomics-for-Border-zone/sn_wc_object_integrated@meta.data.csv', index_col=0)
metadata_sc_t = metadata_sc.rename(index=lambda x: x.split('_')[0])
metadata_sc_t['obs_names'] = metadata_sc_t.index

In [6]:
def get_anndata(mtx_path_id, metadata_orig_id):
    PATH = f'{ROOT_PATH}/{mtx_path_id}'
    ad = sc.read_10x_mtx(PATH)
    ad.obs = ad.obs.rename(index=lambda x: x.split('-')[0])
    A = metadata_sc_t[metadata_sc_t['orig.ident'] == metadata_orig_id]

    ad.obs = ad.obs.merge(A, left_index=True, right_index=True, how='left')

    return ad

In [7]:
def pp_anndata(ad, n_top_genes=None, plot=False):
    ad.var_names_make_unique()

    sc.pp.filter_cells(ad, min_genes=200)
    sc.pp.filter_genes(ad, min_cells=3)

    ad.var['mt'] = ad.var_names.str.startswith('mt-')
    sc.pp.calculate_qc_metrics(ad, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    if plot:
        sc.pl.violin(ad, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
                jitter=0.4, multi_panel=True)
        sc.pl.scatter(ad, x='total_counts', y='pct_counts_mt')
        sc.pl.scatter(ad, x='total_counts', y='n_genes_by_counts')

    ad = ad[ad.obs.pct_counts_mt <= 5, :]

    sc.pp.normalize_total(ad, target_sum=1e4)
    sc.pp.log1p(ad)

    if n_top_genes is not None:
        sc.pp.highly_variable_genes(ad, n_top_genes=n_top_genes)
        ad.raw = ad
        ad = ad[:, ad.var.highly_variable]

    if plot:
        sc.pl.highly_variable_genes(ad)

    return ad

## Prepare training and testing dataset

In [8]:
def get_mtx_path_id_metadata_orig_id(problem_key):
    exp_meta_keys = {
        'ref_0_que_1hr': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn1hr_1', 'sn1_1'),
                ('sn1hr_2', 'sn1_2'),
            ]
        },
        'ref_0_que_4hr': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn4hr', '4HR'),
            ]
        },
        'ref_0_que_1': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd1_1', 'D1_MI'),
                ('snd1_2', 'D1_MI_Hrt_nuclei'),
                ('snd1_3', 'D1_IR30_Hrt_nuclei'),
                ('snd1_4', 'WT_IR30_D1_2'),
                ('snd1_5', 'WT_IR30_D1_3'),
            ]
        },
        'ref_0_que_3': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd3_1', 'WT_IR30_D3'),
                ('snd3_2', 'D3F_M_rep1'),
                ('snd3_3', 'D3F_M_rep2'),
            ]
        },
        'ref_0_que_7': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd7_1', 'snD7_1'),
                ('snd7_2', 'snD7_1'),
                ('snd7_3', 'D5_D7'),
            ]
        },
        'ref_0_que_1-7': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn1hr_1', 'sn1_1'),
                ('sn1hr_2', 'sn1_2'),
                ('sn4hr', '4HR'),
                ('snd1_1', 'D1_MI'),
                ('snd1_2', 'D1_MI_Hrt_nuclei'),
                ('snd1_3', 'D1_IR30_Hrt_nuclei'),
                ('snd1_4', 'WT_IR30_D1_2'),
                ('snd1_5', 'WT_IR30_D1_3'),
                ('snd3_1', 'WT_IR30_D3'),
                ('snd3_2', 'D3F_M_rep1'),
                ('snd3_3', 'D3F_M_rep2'),
                ('snd7_1', 'snD7_1'),
                ('snd7_2', 'snD7_1'),
                ('snd7_3', 'D5_D7'),
            ]
        }
    }

    return exp_meta_keys[problem_key]

In [9]:
import anndata as AD

imp_celltypes = ['Ankrd1', 'Xirp2', 'Myh6']


def get_ad(PROBLEM_KEY):

    exp_meta_keys = get_mtx_path_id_metadata_orig_id(PROBLEM_KEY)

    adatas = {}

    for key, item in exp_meta_keys.items():
        for i, (mtx_path_id, metadata_orig_id) in enumerate(item):
            ad = pp_anndata(get_anndata(mtx_path_id, metadata_orig_id))
            ad = ad[ad.obs['final_cluster'].isin(imp_celltypes)]
            dataset_id = f"{key}_{i+1}"
            ad.obs['sample'] = dataset_id
            ad.obs['dataset'] = key
            adatas[f"{key}_{i+1}"] = ad

    ad = AD.concat(adatas, label='batch_key')
    ad.obs_names_make_unique()

    sc.pp.highly_variable_genes(ad, n_top_genes=5000)
    ad = ad[:, ad.var.highly_variable]

    cluster_to_zone = {
        "Ankrd1": "BZ1",
        "Xirp2": "BZ2",
        "Myh6": "RZ",
    }

    ad.obs["zone"] = ad.obs["final_cluster"].map(cluster_to_zone)

    return ad

In [10]:
## Modify to the directory where outputs should be stored
OUT_PATH = "/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/reproducibility/outputs"
metrics_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-metrics-scanorama.csv"
results_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-results-scanorama.csv"

In [12]:
from scpss import scPSS
import time
import numpy as np


with open(metrics_csv_path, 'w') as f:
    print('PROBLEM_KEY', 'seed', 'accuracy', 'precision', 'recall', 'f1', 'auc', 'aupr', 'outlier_ratio_ref', 'outlier_ratio_que', 'time_taken',  sep=',', file=f, flush=True)


    for PROBLEM_KEY in ['ref_0_que_1hr', 'ref_0_que_4hr', 'ref_0_que_1', 'ref_0_que_3', 'ref_0_que_7',  'ref_0_que_1-7']:
        ad = get_ad(PROBLEM_KEY)

        reference_samples = list(ad.obs[ad.obs['dataset'] == 'reference']['sample'].unique())
        query_samples = list(ad.obs[ad.obs['dataset'] == 'query']['sample'].unique())


        for i in range(25):
            print('RUN: ', i)
            seed = i * 100

            start_time = time.perf_counter()

            scpss = scPSS(ad.copy(), 'sample', reference_samples, query_samples)
            import scanpy.external as sce

            sc.pp.pca(scpss.adata)
            sce.pp.scanorama_integrate(scpss.adata, key='sample')

            scpss.__obsm_str__ = 'X_scanorama'

            scpss.find_optimal_parameters()
            scpss.set_distance_and_condition()

            end_time = time.perf_counter()

            ad_que = scpss.adata[scpss.query_mask]

            true_labels = ad_que.obs['zone'].isin(['BZ1', 'BZ2']).to_numpy()
            predicted_labels = ad_que.obs['scpss_condition'].isin(['diseased']).to_numpy()
            dist_que_ref = scpss.adata.obs.loc[scpss.query_mask, 'scpss_distances']
            dist_ref_ref = scpss.adata.obs.loc[scpss.reference_mask, 'scpss_distances']

            time_taken = end_time - start_time


            from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

            accuracy = accuracy_score(true_labels, predicted_labels)
            precision = precision_score(true_labels, predicted_labels)
            recall = recall_score(true_labels, predicted_labels)
            f1 = f1_score(true_labels, predicted_labels)

            print("Accuracy:", accuracy)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1-score:", f1)

            from sklearn.metrics import roc_auc_score, average_precision_score
            from sklearn.metrics import roc_curve, precision_recall_curve

            y_true = true_labels
            y_scores = dist_que_ref

            auc = roc_auc_score(y_true, y_scores)
            print("AUC:", auc)

            aupr = average_precision_score(y_true, y_scores)
            print("AUPR:", aupr)

            outlier_ratio_ref = np.mean(dist_ref_ref > scpss.best_params['threshold'])
            print("Outlier ratio in reference:", outlier_ratio_ref)
            outlier_ratio_que = np.mean(dist_que_ref > scpss.best_params['threshold'])
            print("Outlier ratio in query:", outlier_ratio_que)


            print(PROBLEM_KEY, seed, accuracy, precision, recall, f1, auc, aupr, outlier_ratio_ref, outlier_ratio_que, time_taken, sep=',', file=f, flush=True)


/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = datase

RUN:  0
[[0.         0.93093525 0.22519084 0.03597122 0.01726619]
 [0.         0.         0.87022901 0.10431655 0.0647482 ]
 [0.         0.         0.         0.67175573 0.56679389]
 [0.         0.         0.         0.         0.83311856]
 [0.         0.         0.         0.         0.        ]]
Processing datasets reference_1 <=> reference_2
Processing datasets reference_2 <=> reference_3
Processing datasets query_1 <=> query_2
Processing datasets reference_3 <=> query_1
Processing datasets reference_3 <=> query_2
Processing datasets reference_1 <=> reference_3
Processing datasets reference_2 <=> query_1
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.7071706110914872
Precision: 0.6316359696641387
Recall: 0.5438432835820896
F1-score: 0.5844611528822055
AUC: 0.7165833708093982
AUPR: 0.6639345371898301
Outlier ratio in reference: 0.07112590799031478
Outlier ratio in query: 0.3260332038149064
RUN:  1
[[0.         0.93093525 0.22519084 0.0359

/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = datase

RUN:  0
[[0.         0.91079137 0.29961832 0.09640288]
 [0.         0.         0.86450382 0.06531532]
 [0.         0.         0.         0.62977099]
 [0.         0.         0.         0.        ]]
Processing datasets reference_1 <=> reference_2
Processing datasets reference_2 <=> reference_3
Processing datasets reference_3 <=> query_1
Processing datasets reference_1 <=> reference_3
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.765015015015015
Precision: 0.7401129943502824
Recall: 0.9550425273390036
F1-score: 0.8339522546419098
AUC: 0.9297708083178367
AUPR: 0.9558503950773987
Outlier ratio in reference: 0.03722760290556901
Outlier ratio in query: 0.7972972972972973
RUN:  1
[[0.         0.91079137 0.29961832 0.09640288]
 [0.         0.         0.86450382 0.06531532]
 [0.         0.         0.         0.62977099]
 [0.         0.         0.         0.        ]]
Processing datasets reference_1 <=> reference_2
Processing datasets reference_2 <=>

/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = datase

...

RUN:  0
[[0.         0.91654676 0.25954198 0.0202952  0.17391304 0.13425926]
 [0.         0.         0.79770992 0.00738007 0.126294   0.04166667]
 [0.         0.         0.         0.48664122 0.48091603 0.125     ]
 [0.         0.         0.         0.         0.82656827 0.76851852]
 [0.         0.         0.         0.         0.         0.94907407]
 [0.         0.         0.         0.         0.         0.        ]]
Processing datasets query_2 <=> query_3
Processing datasets reference_1 <=> reference_2
Processing datasets query_1 <=> query_2
Processing datasets reference_2 <=> reference_3
Processing datasets query_1 <=> query_3
Processing datasets reference_3 <=> query_1
Processing datasets reference_3 <=> query_2
Processing datasets reference_1 <=> reference_3
Processing datasets reference_1 <=> query_2
Processing datasets reference_1 <=> query_3
Processing datasets reference_2 <=> query_2
Processing datasets reference_3 <=> query_3
✅ Found Optimal Parameters.
✅ Stored distances an

/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = datase

RUN:  0
[[0.         0.90647482 0.23664122 0.02493766 0.         0.36137931]
 [0.         0.         0.72709924 0.06483791 0.00124688 0.27985612]
 [0.         0.         0.         0.0436409  0.0019084  0.6851145 ]
 [0.         0.         0.         0.         1.         0.03117207]
 [0.         0.         0.         0.         0.         0.31795511]
 [0.         0.         0.         0.         0.         0.        ]]
Processing datasets query_1 <=> query_2
Processing datasets reference_1 <=> reference_2
Processing datasets reference_2 <=> reference_3
Processing datasets reference_3 <=> query_3
Processing datasets reference_1 <=> query_3
Processing datasets query_2 <=> query_3
Processing datasets reference_2 <=> query_3
Processing datasets reference_1 <=> reference_3
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.6801202232717904
Precision: 0.5983017635532332
Recall: 0.875717017208413
F1-score: 0.7109041521148622
AUC: 0.7380929316894408
AU

/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_normalization.py:216: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
<ipython-input-9-35133eff22c8>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = datase

RUN:  0
[[0.00000000e+00 7.25899281e-01 1.73664122e-01 1.05755396e-01
  2.87769784e-02 3.66906475e-02 1.16935484e-01 0.00000000e+00
  2.66906475e-01 0.00000000e+00 1.43884892e-03 5.53505535e-03
  1.53956835e-01 9.72222222e-02 1.87032419e-02 3.74064838e-03
  2.17985612e-01]
 [0.00000000e+00 0.00000000e+00 5.91603053e-01 2.87769784e-02
  1.65467626e-02 2.66187050e-02 8.06451613e-03 0.00000000e+00
  5.37720706e-02 5.03597122e-03 1.15107914e-02 1.29151292e-02
  1.61490683e-01 3.70370370e-02 9.97506234e-03 0.00000000e+00
  1.53237410e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.79389313e-01
  2.27099237e-01 5.53435115e-02 8.06451613e-03 2.00000000e-01
  3.24427481e-02 1.37404580e-01 2.44274809e-01 2.74809160e-01
  2.90076336e-01 7.40740741e-02 1.24688279e-02 1.90839695e-03
  2.19465649e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  8.82087629e-01 1.93298969e-02 4.03225806e-03 6.00000000e-01
  4.01284109e-03 0.00000000e+00 1.98412698e-03 1.29151292e-02
  4.7619

In [13]:
metrics = pd.read_csv(metrics_csv_path)

In [14]:
metrics

,PROBLEM_KEY,seed,accuracy,precision,recall,f1,auc,aupr,outlier_ratio_ref,outlier_ratio_que,time_taken
0,ref_0_que_1hr,0,0.707171,0.631636,0.543843,0.584461,0.716583,0.663935,0.071126,0.326033,20.390012
1,ref_0_que_1hr,100,0.707171,0.631636,0.543843,0.584461,0.716583,0.663935,0.071126,0.326033,19.371773
2,ref_0_que_1hr,200,0.707171,0.631636,0.543843,0.584461,0.716583,0.663935,0.071126,0.326033,20.014039
3,ref_0_que_1hr,300,0.707171,0.631636,0.543843,0.584461,0.716583,0.663935,0.071126,0.326033,19.283202
4,ref_0_que_1hr,400,0.707171,0.631636,0.543843,0.584461,0.716583,0.663935,0.071126,0.326033,20.559754
...,...,...,...,...,...,...,...,...,...,...,...
145,ref_0_que_1-7,2000,0.752331,0.739202,0.740286,0.739744,0.825269,0.817407,0.031780,0.476166,52.325290
146,ref_0_que_1-7,2100,0.752331,0.739202,0.740286,0.739744,0.825269,0.817407,0.031780,0.476166,52.521360
147,ref_0_que_1-7,2200,0.752331,0.739202,0.740286,0.739744,0.825269,0.817407,0.031780,0.476166,52.546781
148,ref_0_que_1-7,2300,0.752331,0.739202,0.740286,0.739744,0.825269,0.817407,0.031780,0.476166,53.420242


In [15]:
import pandas as pd
import numpy as np
from scipy.stats import sem, t

# Confidence level
confidence = 0.95

# Function to calculate mean ± margin
def mean_margin(series, confidence=0.95):
    n = len(series)
    mean = np.mean(series)
    se = sem(series)
    margin = se * t.ppf((1 + confidence) / 2., n - 1)
    return f"{mean:.4f} ± {margin:.4f}"

# Group by 'Group' and calculate mean ± margin for other columns
grouped = metrics.groupby('PROBLEM_KEY')

result = grouped.agg({
    col: lambda x: mean_margin(x, confidence)
    for col in metrics.columns if col != 'PROBLEM_KEY'
})

# Reset index to make it look cleaner
result.reset_index(inplace=True)

In [16]:
result

,PROBLEM_KEY,seed,accuracy,precision,recall,f1,auc,aupr,outlier_ratio_ref,outlier_ratio_que,time_taken
0,ref_0_que_1,1200.0000 ± 303.7976,0.8469 ± 0.0000,0.8520 ± 0.0000,0.8659 ± 0.0000,0.8589 ± 0.0000,0.9242 ± 0.0000,0.9389 ± 0.0000,0.0275 ± 0.0000,0.5470 ± 0.0000,26.4938 ± 0.4188
1,ref_0_que_1-7,1200.0000 ± 303.7976,0.7523 ± 0.0000,0.7392 ± 0.0000,0.7403 ± 0.0000,0.7397 ± 0.0000,0.8253 ± 0.0000,0.8174 ± 0.0000,0.0318 ± 0.0000,0.4762 ± 0.0000,53.1408 ± 0.2811
2,ref_0_que_1hr,1200.0000 ± 303.7976,0.7072 ± 0.0000,0.6316 ± 0.0000,0.5438 ± 0.0000,0.5845 ± 0.0000,0.7166 ± 0.0000,0.6639 ± 0.0000,0.0711 ± 0.0000,0.3260 ± 0.0000,20.0639 ± 0.1671
3,ref_0_que_3,1200.0000 ± 303.7976,0.6398 ± 0.0000,0.5993 ± 0.0000,0.3253 ± 0.0000,0.4217 ± 0.0000,0.6658 ± 0.0000,0.5727 ± 0.0000,0.0599 ± 0.0000,0.2192 ± 0.0000,14.5745 ± 0.1689
4,ref_0_que_4hr,1200.0000 ± 303.7976,0.7650 ± 0.0000,0.7401 ± 0.0000,0.9550 ± 0.0000,0.8340 ± 0.0000,0.9298 ± 0.0000,0.9559 ± 0.0000,0.0372 ± 0.0000,0.7973 ± 0.0000,14.2394 ± 0.1501
5,ref_0_que_7,1200.0000 ± 303.7976,0.6801 ± 0.0000,0.5983 ± 0.0000,0.8757 ± 0.0000,0.7109 ± 0.0000,0.7381 ± 0.0000,0.6501 ± 0.0000,0.0342 ± 0.0000,0.6574 ± 0.0000,18.0617 ± 0.3858


In [17]:
result.to_csv(results_csv_path, index=False)